In [8]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import pairwise_distances
from sklearn.gaussian_process.kernels import RBF
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics import pairwise_distances 
from scipy.spatial import distance 
import math
from sklearn.datasets import load_iris
from sklearn.cluster import SpectralClustering
from sklearn.metrics.cluster import normalized_mutual_info_score

In [9]:
#SSC MODEL
class ssc_model(object):
    def __init__(self, X, affine=False, alpha1=800,  alpha2 = None, thr=0.02, maxIter=200):
        self.alpha1 = alpha1 
        if not alpha2:
            self.alpha2 = alpha1
        else:
            self.alpha2 = alpha2

        self.X = X
        self.affine = affine    
        self.thr = thr
        self.maxIter = maxIter
        self.N = X.shape[1]   # number of samples
        
        self.T = (self.X)
        T1 = np.abs(self.T - np.diag(np.diag(self.T)))
        self.lambda1 = np.min(np.max(T1,axis=1))
        self.mu1 = self.alpha1/self.lambda1
        self.mu2 = self.alpha2 
        self.I = np.eye(self.N,dtype=np.float32)
        self.ones = np.ones((self.N,self.N),dtype=np.float32)
        self.vec1N = np.ones((1,self.N),dtype = np.float32)
        self.err =[]
        
    def computeCmat(self):
        if not self.affine:
            A = np.linalg.inv(self.mu1*self.T + self.mu2*self.I)
            C1 = np.zeros((self.N,self.N),dtype=np.float32)
            Lambda2 = np.zeros((self.N,self.N),dtype=np.float32)
            err = 10*self.thr
            iter1 = 1
            while (err>self.thr)and(iter1<self.maxIter):
                #update Z
                Z = np.dot(A,self.mu1*self.T + self.mu2*(C1 - Lambda2/self.mu2))
                Z = Z - np.diag(np.diag(Z))
                # update C
                tmp_val = np.abs(Z + Lambda2/self.mu2) - (self.ones/self.mu2)
                C2 = np.maximum(0,tmp_val)*np.sign(Z + Lambda2/self.mu2)
                C2 = C2 - np.diag(np.diag(C2))
                # update lagrangian multipliers
                Lambda2 = Lambda2 + self.mu2*(Z-C2)
                # compute errors
                tmp_val = np.abs(Z - C2)
                err = np.max(tmp_val.reshape(-1,1))
                C1 = C2
                iter1 = iter1 +1
                print('the error is = %f' % err)
        else:
            A = np.linalg.inv(self.mu1*self.T + self.mu2*self.I+ self.mu2*self.ones)
            C1 = np.zeros((self.N,self.N),dtype=np.float32)
            Lambda2 = np.zeros((self.N,self.N),dtype=np.float32)
            Lambda3 = np.zeros((1,self.N),dtype=np.float32)
            err1 = 10*self.thr
            err3 = 10*self.thr
            iter1 = 1
            while ((err1>self.thr)or(err3>self.thr))and(iter1<self.maxIter):
                #update Z
                tmp_val = self.mu1*self.T + self.mu2*(C1-Lambda2/self.mu2) + self.mu2*np.dot(self.vec1N.T,(self.vec1N - Lambda3/self.mu2))
                Z = np.dot(A,tmp_val)
                Z = Z - np.diag(np.diag(Z))
                # update C
                tmp_val = np.abs(Z + Lambda2/self.mu2) - (self.ones/self.mu2)
                C2 = np.maximum(0,tmp_val)*np.sign(Z + Lambda2/self.mu2)
                C2 = C2 - np.diag(np.diag(C2))
                # update lagrangian multipliers
                Lambda2 = Lambda2 + self.mu2*(Z-C2)
                Lambda3 = Lambda3 + self.mu2*(np.dot(self.vec1N,Z) - self.vec1N)
                # compute errors
                tmp_val = np.abs(Z - C2)
                err1 = np.max(tmp_val.reshape(-1,1))
                tmp_val = np.abs(np.dot(self.vec1N,Z) - self.vec1N)
                err3 = np.max(tmp_val.reshape(-1,1))
                
                C1 = C2
                iter1 = iter1 + 1
                print('iter1 = %d, the error 1 is = %f and error 2 is %f' % (iter1, err1, err3,C2))
                
        return C2

In [11]:
#IRIS DataSet
iris_data = load_iris()
X=iris_data.data
y=iris_data.target

In [12]:
#Conformal mapping
a=np.zeros(X.shape[1])
for i in range(0,X.shape[1]):
    a[i]=np.dot(X.T[i],np.ones(X.shape[0]))/X.shape[0]
a=np.matrix(a)
a.shape
dists = distance.cdist(X, a, 'euclidean')
R=np.max(dists)
Xf=np.zeros((150,4))
Xe=np.zeros((150))
for i in range(0,150):
    Xe[i]=R*((np.dot(X[i]**2,np.ones(4))-math.pow(R,2))/((np.dot(X[i]**2,np.ones(4)))+math.pow(R,2)))
    Xf[i]=R*(2*R/((np.dot(X[i]**2,np.ones(4)))+math.pow(R,2))*X[i])
    
X1=np.column_stack((Xf, Xe)) 


In [13]:

A=np.dot(X1,X1.T)/(R**2)  
for i in range(0,X1.shape[0]):
        A[i][i]=int(1)
for i in range(0,X1.shape[0]):
    for j in range(0,X1.shape[0]):
        if A[i,j]>1:
            A[i][j]=int(1)          
A=R*np.arccos(A)
W=csr_matrix((X1.shape[0], X1.shape[0])).toarray()
for i in range(W.shape[0]):
    for j in range(W.shape[0]):
         W[i][j] =np.exp(-(A[i,j]**2)/(np.mean(A)))     
X=(W+W.T)/2

In [14]:
ssc=ssc_model(X)
C=ssc.computeCmat()
sc = SpectralClustering(n_clusters=3, affinity='precomputed', n_init=50, n_jobs=-1)
sc.fit(np.abs(C) + np.abs(C.T))

the error is = 0.001250


SpectralClustering(affinity='precomputed', n_clusters=3, n_init=50, n_jobs=-1)

In [15]:

print("NMI = " + str(normalized_mutual_info_score(y,sc.labels_,average_method='arithmetic')))

NMI = 0.849780562488021
